<a href="https://colab.research.google.com/github/bhp99/devtraining-needit-tokyo/blob/main/Description_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install accelerate

In [20]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-eh3lu9o_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-eh3lu9o_
  Resolved https://github.com/huggingface/transformers to commit d6392482bd8af0d28903b5531ed04bd7273931a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8148049 sha256=4d7f64b69930b715eb62c5c56154e1e526c4c5449d3e13c6b62a57634384ef39
  Stored in directory: /tmp/pip-ephem-wheel-cache-39e103yi/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

# Hugging Face model_path
model_path = 'psmathur/orca_mini_3b'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/21.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [3]:
def generate_text(system, instruction, input=None):

    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"

    tokens = tokenizer.encode(prompt)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to('cuda')

    instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.7, 'generate_len': 1024, 'top_k': 50}

    length = len(tokens[0])
    with torch.no_grad():
        rest = model.generate(
            input_ids=tokens,
            max_length=length+instance['generate_len'],
            use_cache=True,
            do_sample=True,
            top_p=instance['top_p'],
            temperature=instance['temperature'],
            top_k=instance['top_k']
        )
    output = rest[0][length:]
    string = tokenizer.decode(output, skip_special_tokens=True)
    return f'[!] Response: {string}'

In [4]:
import pandas as pd

In [5]:
data = pd.read_excel('./subprocess.xlsx',sheet_name='LINUX')
data

,SUB PROCESS NAME
0,MANAGEMENT REPORT
1,DATA CLEANUP
2,DATA ARCHIVAL
3,DATA MOVEMENT
4,RESTORE DATA
...,...
61,SERVICE LEVEL MONITORING
62,CONFIGURATION ITEM CHANGE
63,CONFIGURATION ITEM DELETE
64,SERVICE LEVEL MONITORING


In [15]:
system = 'You are an AI assistant expierence in IT operations.'
data1 = pd.DataFrame(columns = ["SP_Description"])
print("Started generating descriptions .......")
for i in data.index:

      #print(data.loc[i]['DOMAIN'])
      prompt = "explain breifly in four sentences about "+data.loc[i]['SUB PROCESS NAME'] + " process in Linux"
      print("Generating ans for ......",prompt)
      response = generate_text(system, prompt)
      print(response)
      data1.loc[i] =response

print("Generated all descriptions.......")
final_df = pd.concat([data,data1],axis=1)

Started generating descriptions .......
Generating ans for ...... explain breifly in four sentences about MANAGEMENT REPORT process in Linux
[!] Response: A management report is a document that provides an overview of a company's performance, financial results, and other important information for decision-making purposes. In Linux, the management report process involves gathering data from various sources and analyzing it to generate insights that can be used to make informed decisions. The process involves using tools such as spreadsheets, databases, and reporting software to organize and present the data in a clear and concise manner. Finally, the management report should be reviewed and approved by key stakeholders before it is submitted for review and distribution.
Generating ans for ...... explain breifly in four sentences about DATA CLEANUP process in Linux
[!] Response: Data Cleanup is a process in Linux that involves locating and removing unnecessary or unwanted files from the 

In [17]:
#final_df
final_df.to_excel("linux_sp.xlsx")

In [ ]:
_